# **Running MD Simulations in Python using OpenMM**

In this notebook, we'll see how to run MD simulations using the OpenMM package, and analyze & view our simulation data using MDAnalysis and nglview.

In [ ]:
### This is where we install any packages we need that Google Colab doesn't have by default ###
#install dependencies
!python --version
!wget https://bootstrap.pypa.io/get-pip.py
!python get-pip.py
!rm get-pip.py
!pip install --upgrade MDAnalysis
!pip install ipywidgets
!pip install nglview
!pip install chardet
!pip install archspec
!pip install -q condacolab

import sys
from sys import stdout
sys.path.append('/usr/local/lib/python3.11/site-packages/')

In [ ]:
import condacolab
condacolab.install() # expect a kernel restart

In [ ]:
!mamba install -y -q -c conda-forge openmm pdbfixer

In [ ]:
#load dependencies
from google.colab import output
output.enable_custom_widget_manager()

from openmm.app import *
from openmm import *
from openmm.unit import *
import os
import numpy as np
import MDAnalysis as md
import nglview as ngl
from nglview.adaptor import FileStructure
import scipy.cluster.hierarchy
from scipy.spatial.distance import squareform
import scipy.stats as stats
import matplotlib.pyplot as plt
import pandas as pd
from scipy.interpolate import griddata
import seaborn as sb
from statistics import mean, stdev
from matplotlib import colors
from IPython.display import set_matplotlib_formats

In [ ]:
# setting the path to our folder so we don't have to type this out every single time we bring up files- just append 'drive_path'
drive_path = '/content/drive/MyDrive/Mol_Sim_S25/'
pdbfile1 = drive_path+'folded.pdb'
pdbfile2 = drive_path+'disorder.pdb'
pdbfile3 = drive_path+'polyGLY.pdb'

We have four Protein Data Bank (PDB) files. Run the below code to see an example of what they look like:

In [ ]:
# these are the first 10 lines of the polyGLY pdb file
with open(pdbfile3) as file:
  lines = file.readlines()[:10]
for i in lines:
  print(i)

Using nglview, we can also look at the protein. You can even zoom in/out and rotate the system.



In [ ]:
u = md.Universe(pdbfile3)
ngl.show_mdanalysis(u)

In [ ]:
pdb = PDBFile(pdbfile3) #pick the topology (i.e, structure)
ff = ForceField('charmm36.xml') #pick the forcefield. there are a lot of built-in options!

### 0. Define the system
system = ff.createSystem(pdb.topology, nonbondedMethod=CutoffNonPeriodic)

### 1. Choose parameters of the experiment: temperature, pressure, box size, boundary conditions, etc
temperature= 298*kelvin
frictionCoeff = 1/picosecond
time_step = 0.002*picoseconds
total_steps = 100*picoseconds / time_step

### 2. Choose an algorithm (integrator)
integrator = LangevinIntegrator(temperature, frictionCoeff, time_step)

### 3. Setup the simulation to run, saving coordinates time to time:

### 3a. Create a simulation object
simulation = Simulation(pdb.topology, system, integrator)
simulation.context.setPositions(pdb.positions)

### 3b. Minimize energy
simulation.minimizeEnergy()

### 3c. Save coordinates to dcd file and energies to standard output console:
simulation.reporters.append(DCDReporter(drive_path+'polyGLY.dcd', 1000))
simulation.reporters.append(StateDataReporter(stdout, 5000, step=True, potentialEnergy=True,\
                                                temperature=True, progress=True, totalSteps = total_steps))

### 4. Run!
simulation.step(total_steps)

Now let's visualize the trajectory of our simulation:

In [ ]:
sys = md.Universe(pdbfile3, drive_path+'polyGLY.dcd')
ngl.show_mdanalysis(sys, gui=True)

MDAnalysis can give us a lot of information about our trajectory. For example, let's plot end-to-end distance over time to visualize the protein folding process:

In [ ]:
# define the terminal atoms
N_term = sys.select_atoms('resid 1 and name N')
C_term = sys.select_atoms('resid 25 and name C')

# while going through the trajectory, calculate the distance between these at every frame
distances = []
for frame in sys.trajectory:
  distances.append(np.linalg.norm(N_term.positions - C_term.positions))

distances = np.array(distances)

In [ ]:
# here comes the iconic python plotting library
plt.figure(figsize=(5, 5))
plt.plot(distances, '-k')
plt.xlabel('Timestep')
plt.ylabel('End-to-end distance')
plt.show()

#Exercise 1: Run simulations

In the same folder, there are PDBs of a folded and disordered protein. Run them for **50** picoseconds at two different temperatures: T1 = 100K and T2 = 500K. Make sure to change the names of your input (pdb) and output (dcd) files to distinguish between each run!

In [ ]:
# folded, 100K


In [ ]:
# folded, 500K


In [ ]:
# disordered, 100K
# for disordered protein, use the force field: amber19/protein.ff19SB.xml
pdb = PDBFile(pdbfile2) #pick the topology (i.e, structure)
ff = ForceField('amber19/protein.ff19SB.xml') #pick the forcefield

### 0. Define the system
system = ff.createSystem(pdb.topology, nonbondedMethod=CutoffNonPeriodic)

### 1. Choose parameters of the experiment: temperature, pressure, box size, boundary conditions, etc
temperature= 100*kelvin
frictionCoeff = 1/picosecond
time_step = 0.002*picoseconds
total_steps = 50*picoseconds / time_step

### 2. Choose an algorithm (integrator)
integrator = LangevinIntegrator(temperature, frictionCoeff, time_step)

### 3. Setup the simulation to run, saving coordinates time to time:

### 3a. Create a simulation object
simulation = Simulation(pdb.topology, system, integrator)
simulation.context.setPositions(pdb.positions)

### 3b. Minimize energy
simulation.minimizeEnergy()

### 3c. Save coordinates to dcd file and energies to standard output console:
simulation.reporters.append(DCDReporter(drive_path+'disorder_100K.dcd', 1000))
simulation.reporters.append(StateDataReporter(stdout, 5000, step=True, potentialEnergy=True,\
                                                temperature=True, progress=True, totalSteps = total_steps))

### 4. Run!
simulation.step(total_steps)

In [ ]:
# disordered, 500K
pdb = PDBFile(pdbfile2) #pick the topology (i.e, structure)
ff = ForceField('amber19/protein.ff19SB.xml') #pick the forcefield

### 0. Define the system
system = ff.createSystem(pdb.topology, nonbondedMethod=CutoffNonPeriodic)

### 1. Choose parameters of the experiment: temperature, pressure, box size, boundary conditions, etc
temperature= 500*kelvin
frictionCoeff = 1/picosecond
time_step = 0.002*picoseconds
total_steps = 50*picoseconds / time_step

### 2. Choose an algorithm (integrator)
integrator = LangevinIntegrator(temperature, frictionCoeff, time_step)

### 3. Setup the simulation to run, saving coordinates time to time:

### 3a. Create a simulation object
simulation = Simulation(pdb.topology, system, integrator)
simulation.context.setPositions(pdb.positions)

### 3b. Minimize energy
simulation.minimizeEnergy()

### 3c. Save coordinates to dcd file and energies to standard output console:
simulation.reporters.append(DCDReporter(drive_path+'disorder_500K.dcd', 1000))
simulation.reporters.append(StateDataReporter(stdout, 5000, step=True, potentialEnergy=True,\
                                                temperature=True, progress=True, totalSteps = total_steps))

### 4. Run!
simulation.step(total_steps)

# Exercise 2: More analysis

The MDanalysis package also contains a function for calculating the [radius of gyration](https://userguide.mdanalysis.org/1.1.1/examples/analysis/custom_trajectory_analysis.html#Radius-of-gyration), which is essentially a measure of how compact the protein is. Perform this analysis on the trajectories you obtained in Exercise 1.

For help with plotting: check out the [matplotlib](https://matplotlib.org/) documentation.

In [ ]:
from MDAnalysis.analysis.base import (AnalysisBase,
                                      AnalysisFromFunction,
                                      analysis_class)

def radgyr(atomgroup, masses, total_mass=None):
    # coordinates change for each frame
    coordinates = atomgroup.positions
    center_of_mass = atomgroup.center_of_mass()

    # get squared distance from center
    ri_sq = (coordinates-center_of_mass)**2
    # sum the unweighted positions
    sq = np.sum(ri_sq, axis=1)
    sq_x = np.sum(ri_sq[:,[1,2]], axis=1) # sum over y and z
    sq_y = np.sum(ri_sq[:,[0,2]], axis=1) # sum over x and z
    sq_z = np.sum(ri_sq[:,[0,1]], axis=1) # sum over x and y

    # make into array
    sq_rs = np.array([sq, sq_x, sq_y, sq_z])

    # weight positions
    rog_sq = np.sum(masses*sq_rs, axis=1)/total_mass
    # square root and return
    return np.sqrt(rog_sq)

In [ ]:
# folded, 500K


In [ ]:
# disordered, 500K


# Exercise 3 (optional): Even more analysis

MDanalysis has a lot more packages for analysis, like outputting dihedral angles and hydrogen bonds over time. Feel free to try them out!

https://userguide.mdanalysis.org/stable/examples/analysis/README.html